In [1]:
import skimage.io
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import cv2
from skimage import exposure, color
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
def calcular_promedio_color(imagen):

    promedio_color = imagen.mean(axis=(0,1))
    std = imagen.std(axis=(0,1))

    data = {
        'intensidad_Rojo': promedio_color[0], 'contraste_Rojo': std[0],
        'intensidad_Verde': promedio_color[1], 'contraste_Verde': std[1],
        'intensidad_Azul': promedio_color[2], 'contraste_Azul': std[2]
    }

    df = pd.DataFrame(data, index=[0])

    return df

In [3]:
def media_imagen_threshold(imagen):
    img_gray = color.rgb2gray(imagen)
    img_ampliacion_rango = exposure.rescale_intensity(img_gray, in_range='image', out_range=(0, 1))
    smoothed_image = cv2.GaussianBlur(img_ampliacion_rango, (5, 5), 0)
    imagen_umbralizada_globulos_blancos = (smoothed_image > 0.25).astype(np.uint8) * 255
    imagen_umbralizada_globulos_negros = 255 - imagen_umbralizada_globulos_blancos
    media_negro = np.mean(imagen_umbralizada_globulos_negros)
    std_negro = np.std(imagen_umbralizada_globulos_negros)
    return media_negro, std_negro

In [4]:
carpeta = 'data'
lista_dfs_res = []

for carpetita in os.listdir(carpeta):
    ruta_carpetita = os.path.join(carpeta,carpetita)
    resultados = []

    for nombre_archivo in os.listdir(ruta_carpetita):
        ruta_imagen = os.path.join(ruta_carpetita, nombre_archivo)

        if os.path.isfile(ruta_imagen) and nombre_archivo.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            imagen = skimage.io.imread(ruta_imagen)
            
            df_resultado = calcular_promedio_color(imagen)
            media_binario, std_binario = media_imagen_threshold(imagen) 
            df_resultado['media_binario'] = media_binario
            df_resultado['std_binario'] = std_binario
            resultados.append(df_resultado)

    df_res = pd.concat(resultados, ignore_index=True)
    if carpetita == 'healthy':
        df_res['label'] = 0
    else:
        df_res['label'] = 1


    lista_dfs_res.append(df_res)

df = pd.concat([lista_dfs_res[0], lista_dfs_res[1]], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)

In [12]:
df.to_csv('dataset_features.csv', index=False)
print(df.head())

   intensidad_Rojo  contraste_Rojo  intensidad_Verde  contraste_Verde  \
0       150.256180        9.135768        132.956495        19.052737   
1       146.470863       10.583887        137.198766        11.687097   
2       146.733711       10.955865        138.765872        13.009746   
3       160.149168       12.170018        146.903018        20.769447   
4       144.689658        9.840177        131.922146        15.950798   

   intensidad_Azul  contraste_Azul  media_binario  std_binario  label  
0       118.456617       15.511611       0.798272    14.245071      0  
1       129.107212       10.766829       0.301617     8.764773      0  
2       131.253735       12.633971       0.351059     9.454994      0  
3       138.231599       11.768046       2.040163    22.717377      1  
4       120.354844       12.300901       0.542454    11.748685      0  


In [8]:
X = df.drop('label', axis=1)
y = df['label']

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.2, random_state=42)

clasificador = RandomForestClassifier()

clasificador.fit(X_train, y_train)
pred = clasificador.predict(X_val)

accuracy = accuracy_score(y_val, pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 1.0000


In [6]:
coefs = pd.Series(clasificador.coef_[0], index=X.columns)

coefs_sorted = coefs.abs().sort_values(ascending=False)

print("Variables ordenadas por su aportación al modelo:")
print(coefs_sorted)

Variables ordenadas por su aportación al modelo:
intensidad_Azul     0.580609
std_binario         0.502024
contraste_Verde     0.358404
intensidad_Rojo     0.205283
intensidad_Verde    0.205222
contraste_Azul      0.152689
contraste_Rojo      0.135927
media_binario       0.074024
dtype: float64


In [9]:
X = df.drop('label', axis=1)
X = X[coefs_sorted.index[:4]]
y = df['label']

X_train_total, X_test, y_train_total, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_total, y_train_total, test_size=0.2, random_state=42)

clasificador = RandomForestClassifier()

clasificador.fit(X_train, y_train)
pred = clasificador.predict(X_val)

accuracy = accuracy_score(y_val, pred)
print(f"Precisión del modelo: {accuracy:.4f}")

Precisión del modelo: 1.0000
